In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                if output[col].dtypes == 'object' :
                    output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                if output[colname].dtypes == 'object' :
                    output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [66]:
#read the data
tdata = pd.read_csv(r"E:\AV\train_u6lujuX.csv")
test = pd.read_csv(r"E:\AV\test_Y3wMUE5.csv")


#handling missing data for trainining
tdata.Gender.fillna("Unknown",inplace=True)
tdata.Married.fillna("Unknown",inplace=True)
tdata.Dependents.fillna('Unknown',inplace=True)
tdata.Self_Employed.fillna('No',inplace=True)
tdata.LoanAmount.fillna(np.mean(tdata.LoanAmount),inplace=True)
tdata.Loan_Amount_Term.fillna(np.mean(tdata.Loan_Amount_Term),inplace=True)
tdata.Credit_History.fillna(1,inplace=True)
tdata['Final_Inc'] = tdata.ApplicantIncome + tdata.CoapplicantIncome
tdata['Ratio'] = tdata.LoanAmount / tdata.Final_Inc

#handling missing data for test
test.Gender.fillna("Unknown",inplace=True)
test.Married.fillna("Unknown",inplace=True)
test.Dependents.fillna('Unknown',inplace=True)
test.Self_Employed.fillna('No',inplace=True)
test.LoanAmount.fillna(np.mean(tdata.LoanAmount),inplace=True)
test.Loan_Amount_Term.fillna(np.mean(tdata.Loan_Amount_Term),inplace=True)
test.Credit_History.fillna(1,inplace=True)
test['Final_Inc'] = test.ApplicantIncome + test.CoapplicantIncome
test['Ratio'] = test.LoanAmount / test.Final_Inc


#Binning the continiuous variables
#tdata['LoanAmount'] = pd.cut(tdata['LoanAmount'],10,labels = ['1','2','3','4','5','6','7','8','9','10'])
#tdata['ApplicantIncome'] = pd.cut(tdata['ApplicantIncome'],10,labels = ['1','2','3','4','5','6','7','8','9','10'])
#tdata['CoapplicantIncome'] = pd.cut(tdata['CoapplicantIncome'],10,labels = ['1','2','3','4','5','6','7','8','9','10'])
#tdata['Loan_Amount_Term'] = pd.cut(tdata['Loan_Amount_Term'],10,labels = ['1','2','3','4','5','6','7','8','9','10'])

#Label encoding
tdata_fin = MultiColumnLabelEncoder(columns = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Credit_History','Loan_Status']).fit_transform(tdata)

test_fin = MultiColumnLabelEncoder(columns = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Credit_History']).fit_transform(test)

In [67]:
#Spliting data and target
x_train_value = tdata_fin.drop(['Loan_ID','Loan_Status'],axis = 1).as_matrix()
x_train_target = tdata_fin['Loan_Status'].as_matrix()


x_test_value = test_fin.drop(['Loan_ID'],axis = 1).as_matrix()

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold,cross_val_score

kf = KFold(n = x_train_value.shape[0],n_folds = 10)

for k in xrange(500,1000,50) :
    for i in [3,4] :
        forest = RandomForestClassifier(n_estimators = k,criterion ='entropy',max_features = i)
        print k
        print cross_val_score(forest,x_train_value,x_train_target,cv=10).mean()
    

500
0.796539162113
500
0.791673172001
550
0.794899817851
550
0.798178506375
600
0.796539162113
600
0.793312516263
650
0.793260473588
650
0.796591204788
700
0.796539162113
700
0.79823054905
750
0.791621129326
750
0.794899817851
800
0.794899817851
800
0.793312516263
850
0.799817850638
850
0.796591204788
900
0.796539162113
900
0.8014571949
950
0.794899817851
950
0.794951860526


In [68]:
forest = RandomForestClassifier(n_estimators = 900,criterion ='entropy',max_features = 4)
forest_res = forest.fit(x_train_value,x_train_target)

predicted = forest_res.predict(x_test_value)

In [69]:
res = []
for x in predicted :
    if x == 1 :
        res.append("Y")
    else:
        res.append("N")

submission = pd.DataFrame({ 'id': test['Loan_ID'],
                            'status': res })

submission.to_csv(r"E:\AV\ans__1.csv")